# **Importacion de paquetes**
---

In [118]:
import pandas as pd
import numpy as np
import random
import matplotlib.pylab as plt
from matplotlib.ticker import MaxNLocator
import pylab as p
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import KFold,train_test_split
from sklearn.metrics import recall_score,accuracy_score,classification_report,confusion_matrix
from catboost import CatBoostClassifier

import warnings
warnings.filterwarnings("ignore")
pd.options.mode.chained_assignment = None  # default='warn'

with warnings.catch_warnings():
    warnings.filterwarnings("ignore",category=DeprecationWarning)
    from sklearn import model_selection
    import xgboost as xgb

#import xgboost as xgb
import operator
import timeit
import scipy.stats as stats

Entender que producto hay o no segun agrupaciones, por ejemplo grupos de cantidades de renta, id_segmento(universitario, no etx)

# **Preprocesamiento**
---

In [81]:
reader = pd.read_csv('../../Data/dataset_para_modelar.csv')

Sustituimos los valores anómalos

In [82]:
reader.replace(' NA', -1, inplace=True)
reader.replace('         NA', -1, inplace=True)
reader.fillna(-1, inplace=True)

Diccionarios para mapear:

In [83]:
emp_dict = {'N':0,-1:-1,'A':1,'B':2,'F':3,'S':4}
inreaderall_dict = {'N':0,-1:-1,'S':1}
sexo_dict = {'V':0,'H':1,-1:-1}
tiprel_dict = {'A':0,-1:-1,'I':1,'P':2,'N':3,'R':4}
indresi_dict = {'N':0,-1:-1,'S':1}
indext_dict = {'N':0,-1:-1,'S':1}
conyuemp_dict = {'N':0,-1:-1,'S':1}
segmento_dict = {-1:-1,'01 - TOP':1,'02 - PARTICULARES':2,'03 - UNIVERSITARIO':3}

Mapeo de valores y conversion de los tipos

In [84]:
reader['cod_persona'] = reader['cod_persona'].astype(np.int32)
reader['imp_renta'] = reader['imp_renta'].astype(np.int32)
#sustituimos el 99 por 2 para que sean valores mas cercanos
reader['xti_rel'] = reader['xti_rel'].map(lambda x: 2 if x == 99 else x).astype(np.int8)

reader['xti_empleado'] = reader['xti_empleado'].map(lambda x: emp_dict[x]).astype(np.int8)
reader['sexo'] = reader['sexo'].map(lambda x: sexo_dict[x]).astype(np.int8)
reader['edad'] = reader['edad'].astype(np.int16)

In [85]:
reader['xti_nuevo_cliente'] = reader['xti_nuevo_cliente'].astype(np.int8)
reader['num_antiguedad'] = reader['num_antiguedad'].map(lambda x: -1 if x == '     NA' else x).astype(int)
reader['num_antiguedad'] = reader['num_antiguedad'].map(lambda x: -1 if x == -999999 else x).astype(np.int16)
reader['xti_rel_1mes'] = reader['xti_rel_1mes'].map(lambda x: -2 if x == 'P' else x).astype(np.float16)
reader['xti_rel_1mes'] = reader['xti_rel_1mes'].astype(np.int8)

In [86]:
reader['tip_rel_1mes'] = reader['tip_rel_1mes'].map(lambda x: tiprel_dict[x]).astype(np.int8)
reader['indresi'] = reader['indresi'].map(lambda x: indresi_dict[x]).astype(np.int8)
reader['indext'] = reader['indext'].map(lambda x: indext_dict[x]).astype(np.int8)

Convertimos des canal a numerico

In [87]:
canal_dict={value:idx+1  for idx, value in enumerate(reader['des_canal'].unique())}
reader['des_canal'] = reader['des_canal'].map(lambda x: canal_dict[x]).astype(np.int16)

In [88]:
pais_dict = {'LV': 102,'CA': 2,'GB': 9,'EC': 19,'BY': 64,'ML': 104,'MT': 118,
 'LU': 59,'GR': 39,'NI': 33,'BZ': 113,'QA': 58,'DE': 10,'AU': 63,'IN': 31,
 'GN': 98,'KE': 65,'HN': 22,'JM': 116,'SV': 53,'TH': 79,'IE': 5,'TN': 85,
 'PH': 91,'ET': 54,'AR': 13,'KR': 87,'GA': 45,'FR': 8,'SG': 66,'LB': 81,
 'MA': 38,'NZ': 93,'SK': 69,'CN': 28,'GI': 96,'PY': 51,'SA': 56,'PL': 30,
 'PE': 20,'GE': 78,'HR': 67,'CD': 112,'MM': 94,'MR': 48,'NG': 83,'HU': 106,
 'AO': 71,'NL': 7,'GM': 110,'DJ': 115,'ZA': 75,'OM': 100,'LT': 103,'MZ': 27,
 'VE': 14,'EE': 52,'CF': 109,'CL': 4,'SL': 97,'DO': 11,'PT': 26,'ES': 0,
 'CZ': 36,'AD': 35,'RO': 41,'TW': 29,'BA': 61,'IS': 107,'AT': 6,'ZW': 114,
 'TR': 70,'CO': 21,'PK': 84,'SE': 24,'AL': 25,'CU': 72,'UY': 77,'EG': 74,'CR': 32,
 'GQ': 73,'MK': 105,'KW': 92,'GT': 44,'CM': 55,'SN': 47,'KZ': 111,'DK': 76,
 'LY': 108,'AE': 37,'PA': 60,'UA': 49,'GW': 99,'TG': 86,'MX': 16,'KH': 95,
 'FI': 23,'NO': 46,'IT': 18,'GH': 88, 'JP': 82,'RU': 43,'PR': 40,'RS': 89,
 'DZ': 80,'MD': 68,-1: 1,'BG': 50,'CI': 57,'IL': 42,'VN': 90,'CH': 3,'US': 15,'HK': 34,
 'CG': 101,'BO': 62,'BR': 17,'BE': 12,'BM': 117}

In [89]:
reader['xti_extra'] = reader['xti_extra'].map(lambda x: inreaderall_dict[x]).astype(np.int8)
reader['pais'] = reader['pais'].map(lambda x: pais_dict[x]).astype(np.int8)
reader['tip_dom'] = reader['tip_dom'].astype(np.int8)
reader['cod_provincia'] = reader['cod_provincia'].astype(np.int8)

In [90]:
reader['xti_actividad_cliente'] = reader['xti_actividad_cliente'].astype(np.int8)
reader['fecha_dato_month'] = reader['mes'].map(lambda x: int(x[5:7])).astype(np.int8)
reader['fecha_dato_year'] = reader['mes'].map(lambda x: int(x[0:4]) - 2015).astype(np.int8)
reader['month_int'] = (reader['fecha_dato_month'] + 12 * reader['fecha_dato_year']).astype(np.int8)
reader.drop('mes',axis=1,inplace=True)

In [91]:
reader['fecha1'] = reader['fecha1'].map(lambda x: '2020-01-01' if x == -1 else x)
reader['fecha_alta_month'] = reader['fecha1'].map(lambda x: int(x[5:7])).astype(np.int16)
reader['fecha_alta_year'] = reader['fecha1'].map(lambda x: int(x[0:4]) - 1995).astype(np.int16)

In [92]:
reader['fecha_alta_day'] = reader['fecha1'].map(lambda x: int(x[8:10])).astype(np.int16)
reader['fecha_alta_month_int'] = (reader['fecha_alta_month'] + 12 * reader['fecha_alta_year']).astype(np.int16)
reader['fecha_alta_day_int'] = (reader['fecha_alta_day'] + 30 * reader['fecha_alta_month'] + 365 * reader['fecha_alta_year']).astype(np.int32)
reader.drop('fecha1',axis=1,inplace=True)

In [93]:
reader['fec_ult_cli_1t'] = reader['fec_ult_cli_1t'].map(lambda x: '2020-01-01' if x == -1 else x)
reader['ult_fec_cli_1t_month'] = reader['fec_ult_cli_1t'].map(lambda x: int(x[5:7])).astype(np.int16)
reader['ult_fec_cli_1t_year'] = reader['fec_ult_cli_1t'].map(lambda x: int(x[0:4]) - 2015).astype(np.int16)
reader['ult_fec_cli_1t_day'] = reader['fec_ult_cli_1t'].map(lambda x: int(x[8:10])).astype(np.int16)
reader['ult_fec_cli_1t_month_int'] = (reader['ult_fec_cli_1t_month'] + 12 * reader['ult_fec_cli_1t_year']).astype(np.int8)
reader.drop('fec_ult_cli_1t',axis=1,inplace=True)

In [94]:
reader['id_segmento'] = reader['id_segmento'].map(lambda x: segmento_dict[x]).astype(np.int8)
target_cols=[f'ind_prod{i}' for i in range(1,26)]
for col in target_cols:
    reader[col] = reader[col].astype(np.int8)

In [95]:
reader.to_csv('sant_limpio.csv',index=False)

In [96]:
reader.rename(columns={'cod_persona':'id'},inplace=True)

In [97]:
cols_valor_unico=[]
muy_bajos_unos=[]
bajos_unos=[]
medios_unos=[]
for col in target_cols:
    if len(reader[col].value_counts().values)<2:
        cols_valor_unico.append(col)
    elif reader[col].value_counts().values[1]/len(reader) <0.001:
        muy_bajos_unos.append(col)
    elif reader[col].value_counts().values[1]/len(reader) <0.005:
        bajos_unos.append(col)
    else:
        medios_unos.append(col)
print('Columnas de valor unico: ',cols_valor_unico)
print('Columnas con muy poco 1: ',muy_bajos_unos)
print('Columnas con pocos 1: ',bajos_unos)
print('Columnas con intermedio 1: ',medios_unos)

Columnas de valor unico:  ['ind_prod2']
Columnas con muy poco 1:  ['ind_prod1', 'ind_prod4']
Columnas con pocos 1:  ['ind_prod10', 'ind_prod11', 'ind_prod17', 'ind_prod21']
Columnas con intermedio 1:  ['ind_prod3', 'ind_prod5', 'ind_prod6', 'ind_prod7', 'ind_prod8', 'ind_prod9', 'ind_prod12', 'ind_prod13', 'ind_prod14', 'ind_prod15', 'ind_prod16', 'ind_prod18', 'ind_prod19', 'ind_prod20', 'ind_prod22', 'ind_prod23', 'ind_prod24', 'ind_prod25']


In [98]:
reader.columns

Index(['Unnamed: 0', 'id', 'pais', 'sexo', 'edad', 'xti_empleado',
       'xti_nuevo_cliente', 'num_antiguedad', 'xti_rel', 'xti_rel_1mes',
       'tip_rel_1mes', 'indresi', 'indext', 'des_canal', 'xti_extra',
       'tip_dom', 'cod_provincia', 'xti_actividad_cliente', 'imp_renta',
       'id_segmento', 'mean_engagement', 'ind_prod1', 'ind_prod2', 'ind_prod3',
       'ind_prod4', 'ind_prod5', 'ind_prod6', 'ind_prod7', 'ind_prod8',
       'ind_prod9', 'ind_prod10', 'ind_prod11', 'ind_prod12', 'ind_prod13',
       'ind_prod14', 'ind_prod15', 'ind_prod16', 'ind_prod17', 'ind_prod18',
       'ind_prod19', 'ind_prod20', 'ind_prod21', 'ind_prod22', 'ind_prod23',
       'ind_prod24', 'ind_prod25', 'fecha_dato_month', 'fecha_dato_year',
       'month_int', 'fecha_alta_month', 'fecha_alta_year', 'fecha_alta_day',
       'fecha_alta_month_int', 'fecha_alta_day_int', 'ult_fec_cli_1t_month',
       'ult_fec_cli_1t_year', 'ult_fec_cli_1t_day',
       'ult_fec_cli_1t_month_int'],
      dtype='object

In [99]:
for col in target_cols:
    reader[col].replace(-1,0,inplace=True)
data_reducida=reader[['id','month_int'] + target_cols].copy()
data_reducida.fillna(0,inplace=True)

In [100]:
data_reducida.sort_values(by = ['id','month_int'],inplace=True)

In [101]:
#Creamos columnas con un shift sobre un dataset reducido para pruebas
DIFF_CONDS = {}
for shift_val in range(1,18):
    name = 'id_shift_' + str(shift_val)
    data_reducida[name] = data_reducida['id'].shift(shift_val).fillna(0).astype(np.int32)
    DIFF_CONDS[shift_val] = ((data_reducida['id'] - data_reducida[name]) != 0)
    data_reducida.drop(name,axis = 1,inplace=True)

for col in target_cols:
    for shift_val in range(1,18):
        name = col + '_s_' + str(shift_val)
        data_reducida[name] = data_reducida[col].shift(shift_val).fillna(0).astype(np.int8)
        data_reducida[name][DIFF_CONDS[shift_val]] = 0

In [102]:
for col in target_cols:
    data_reducida[col] = (data_reducida[col] - data_reducida[col + '_s_1']).astype(np.int8)
    data_reducida[col] = (data_reducida[col] > 0).astype(np.int8)

In [103]:
MIN_MONTH_DICT = data_reducida.groupby('id')['month_int'].min().to_dict()
data_reducida['min_month_int'] = data_reducida['id'].map(lambda x: MIN_MONTH_DICT[x])

data_reducida = data_reducida[data_reducida['min_month_int'] != data_reducida['month_int']]

data_reducida['sum_inds'] = data_reducida[target_cols].sum(axis=1)
data_reducida = data_reducida[(data_reducida['sum_inds'] != 0) | (data_reducida['month_int'] == 18)].copy()
#combined_small = combined_small[(combined_small['sum_inds'] != 0) | (combined_small['month_int'] >= 17)].copy()
data_reducida.to_csv('data_reducida_procesada.csv', index=False)

In [104]:
cols_to_combine = ['edad', 'num_antiguedad', 'des_canal', 'cod_provincia',
       'fecha_alta_day', 'fecha_alta_month', 'fecha_alta_month_int','fecha_alta_day_int',
       'fecha_alta_year', 'fecha_dato_month', 'fecha_dato_year',
       'xti_actividad_cliente',
       'xti_empleado',
       'xti_nuevo_cliente',
       'indext',
       'xti_extra', 'xti_rel', 'xti_rel_1mes', 'indresi',
       'pais', 'imp_renta', 'id_segmento', 'sexo',
       'tip_rel_1mes', 'ult_fec_cli_1t_day', 'ult_fec_cli_1t_month',
       'ult_fec_cli_1t_month_int', 'ult_fec_cli_1t_year']

#Creamos columnas con un shift sobre el dataset inicial
DIFF_CONDS = {}
for shift_val in [1]:
    name = 'id_shift_' + str(shift_val)
    reader[name] = reader['id'].shift(shift_val).fillna(0).astype(np.int32)
    DIFF_CONDS[shift_val] = ((reader['id'] - reader[name]) != 0)
    reader.drop(name,axis = 1,inplace=True)
shifted_feature_names = []
for col in cols_to_combine + target_cols:
    for shift_val in [1]:
        name = col + '_s_' + str(shift_val)
        reader[name] = reader[col].shift(shift_val).fillna(0).astype(np.int32)
        reader[name][DIFF_CONDS[shift_val]] = 0
        if col in cols_to_combine:
            shifted_feature_names.append(name)

In [105]:
reader=reader.iloc[:,1:]

In [106]:
#Creamos columnas con las varianciones de las variables X para predecir
diff_feautres_s1 = []
for col in cols_to_combine:
    name = col + '_s1_diff'
    diff_feautres_s1.append(name)
    reader[name] = (reader[col] - reader[col + '_s_1']).astype(np.int32)

In [107]:
MIN_MONTH_DICT = reader.groupby('id')['month_int'].min().to_dict()
reader['min_month_int'] = reader['id'].map(lambda x: MIN_MONTH_DICT[x]).astype(np.int8)

#Antiguedad minima del usuario
MIN_ANTIGUEDAD_DICT = reader.groupby('id')['num_antiguedad'].min().to_dict()
reader['min_antiguedad'] = reader['id'].map(lambda x: MIN_ANTIGUEDAD_DICT[x]).astype(np.int16)

#Antiguedad maxima del usuario
MAX_ANTIGUEDAD_DICT = reader.groupby('id')['num_antiguedad'].max().to_dict()
reader['max_antiguedad'] = reader['id'].map(lambda x: MAX_ANTIGUEDAD_DICT[x]).astype(np.int16)

#Edad minima del usuario
MIN_AGE_DICT = reader.groupby('id')['edad'].min().to_dict()
reader['min_edad'] = reader['id'].map(lambda x: MIN_AGE_DICT[x]).astype(np.int16)

#Edad maxima del usuario
MAX_AGE_DICT = reader.groupby('id')['edad'].max().to_dict()
reader['max_edad'] = reader['id'].map(lambda x: MAX_AGE_DICT[x]).astype(np.int16)

#Minimo maximo y desviancion estandar de la renta por cada usuario
MIN_RENTA_DICT = reader.groupby('id')['imp_renta'].min().to_dict()
reader['min_renta'] = reader['id'].map(lambda x: MIN_RENTA_DICT[x])
MAX_RENTA_DICT = reader.groupby('id')['imp_renta'].max().to_dict()
reader['max_renta'] = reader['id'].map(lambda x: MAX_RENTA_DICT[x])

In [108]:
RENTA_VAL_COUNTS = reader.groupby('imp_renta')['id'].nunique().to_dict()
reader['renta_freq'] = reader['imp_renta'].map(lambda x: RENTA_VAL_COUNTS[x])
reader.sort_values(by = ['id','month_int'],inplace=True)
#Eliminamos los id duplicados porque hemos reducido la informacion a una sola fila por usuario
combined_nd = reader.drop_duplicates('id')

In [124]:
prod='ind_prod5'
columnas_sin_target=combined_nd.columns
columnas_sin_target=[col for col in columnas_sin_target if col not in target_cols]
X=combined_nd[columnas_sin_target].copy()
y=combined_nd[prod]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Crear el modelo CatBoostClassifier
model = CatBoostClassifier(
    learning_rate=0.05,
    iterations=1500,
    depth=6,
    loss_function='Logloss',
    verbose=100)

# Entrenar el modelo
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

'''
# Evaluar la precisión del modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
recall = recall_score(y_test, y_pred)
print(f'Recall: {recall}')
conf = confusion_matrix(y_test, y_pred)
print(f'Matriz de confusion: {conf}')
'''


0:	learn: 0.5887254	total: 46.4ms	remaining: 1m 9s
100:	learn: 0.1502964	total: 1.73s	remaining: 23.9s
200:	learn: 0.1444607	total: 3.31s	remaining: 21.4s
300:	learn: 0.1390917	total: 4.65s	remaining: 18.5s
400:	learn: 0.1346807	total: 5.96s	remaining: 16.3s
500:	learn: 0.1308215	total: 7.26s	remaining: 14.5s
600:	learn: 0.1272773	total: 8.57s	remaining: 12.8s
700:	learn: 0.1236879	total: 9.88s	remaining: 11.3s
800:	learn: 0.1203194	total: 11.3s	remaining: 9.83s
900:	learn: 0.1172622	total: 12.6s	remaining: 8.39s
1000:	learn: 0.1143613	total: 14s	remaining: 6.96s
1100:	learn: 0.1115751	total: 15.3s	remaining: 5.55s
1200:	learn: 0.1088844	total: 16.6s	remaining: 4.14s
1300:	learn: 0.1061771	total: 18s	remaining: 2.75s
1400:	learn: 0.1036014	total: 19.3s	remaining: 1.36s
1499:	learn: 0.1010327	total: 20.7s	remaining: 0us


"\n# Evaluar la precisión del modelo\naccuracy = accuracy_score(y_test, y_pred)\nprint(f'Accuracy: {accuracy}')\nrecall = recall_score(y_test, y_pred)\nprint(f'Recall: {recall}')\nconf = confusion_matrix(y_test, y_pred)\nprint(f'Matriz de confusion: {conf}')\n"

In [130]:
model.predict_proba(X_test)

array([[9.99913556e-01, 8.64440090e-05],
       [9.98094371e-01, 1.90562903e-03],
       [9.92519828e-01, 7.48017181e-03],
       ...,
       [9.99606505e-01, 3.93495354e-04],
       [9.99641377e-01, 3.58622820e-04],
       [9.99999620e-01, 3.80464493e-07]])

In [132]:
list(y_test)

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [111]:
clf=CatBoostClassifier()

['ind_prod1',
 'ind_prod2',
 'ind_prod3',
 'ind_prod4',
 'ind_prod5',
 'ind_prod6',
 'ind_prod7',
 'ind_prod8',
 'ind_prod9',
 'ind_prod10',
 'ind_prod11',
 'ind_prod12',
 'ind_prod13',
 'ind_prod14',
 'ind_prod15',
 'ind_prod16',
 'ind_prod17',
 'ind_prod18',
 'ind_prod19',
 'ind_prod20',
 'ind_prod21',
 'ind_prod22',
 'ind_prod23',
 'ind_prod24',
 'ind_prod25']

In [73]:
reader.head()

,id,pais,sexo,edad,xti_empleado,xti_nuevo_cliente,num_antiguedad,xti_rel,xti_rel_1mes,tip_rel_1mes,...,indresi_s1_diff,pais_s1_diff,imp_renta_s1_diff,id_segmento_s1_diff,sexo_s1_diff,tip_rel_1mes_s1_diff,ult_fec_cli_1t_day_s1_diff,ult_fec_cli_1t_month_s1_diff,ult_fec_cli_1t_month_int_s1_diff,ult_fec_cli_1t_year_s1_diff
0,178103,0,1,35,0,0,6,1,1,0,...,1,0,87218,2,1,0,1,1,61,5
1,503082,0,0,27,0,0,35,1,1,1,...,1,0,70777,3,0,1,1,1,61,5
2,502996,0,0,37,0,0,35,1,1,0,...,1,0,104035,2,0,0,1,1,61,5
3,503053,0,1,23,0,0,35,1,1,0,...,1,0,136930,3,1,0,1,1,61,5
4,503031,0,1,44,0,0,35,1,1,1,...,1,0,110245,2,1,1,1,1,61,5


In [19]:
reader.columns

Index(['Unnamed: 0', 'cod_persona', 'mes', 'pais', 'sexo', 'edad', 'fecha1',
       'xti_empleado', 'xti_nuevo_cliente', 'num_antiguedad', 'xti_rel',
       'fec_ult_cli_1t', 'xti_rel_1mes', 'tip_rel_1mes', 'indresi', 'indext',
       'des_canal', 'xti_extra', 'tip_dom', 'cod_provincia',
       'xti_actividad_cliente', 'imp_renta', 'id_segmento', 'mean_engagement',
       'ind_prod1', 'ind_prod2', 'ind_prod3', 'ind_prod4', 'ind_prod5',
       'ind_prod6', 'ind_prod7', 'ind_prod8', 'ind_prod9', 'ind_prod10',
       'ind_prod11', 'ind_prod12', 'ind_prod13', 'ind_prod14', 'ind_prod15',
       'ind_prod16', 'ind_prod17', 'ind_prod18', 'ind_prod19', 'ind_prod20',
       'ind_prod21', 'ind_prod22', 'ind_prod23', 'ind_prod24', 'ind_prod25'],
      dtype='object')